## Notebook with SPARQL Kernel

To run this notebook, install the [SPARQL kernel](https://github.com/paulovn/sparql-kernel):
```
pip install sparqlkernel
jupyter sparqlkernel install
```

To use Graphviz to draw graphs, you should also install Pydot and Graphviz:
```
conda install pydot graphviz
```
Set the path to the dot script in your PATH variable. For Miniconda users on Windows, the path would be like `C:\Users\user\Miniconda3\Library\bin\graphviz`

In [1]:
%endpoint https://data-issa.cirad.fr/sparql
%display table
%show 20

%prefix rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
%prefix owl:    <http://www.w3.org/2002/07/owl#>
%prefix xsd:    <http://www.w3.org/2001/XMLSchema#>

%prefix agrovoc: <http://aims.fao.org/aos/agrovoc/>
%prefix dce:    <http://purl.org/dc/elements/1.1/>
%prefix dct:    <http://purl.org/dc/terms/>
%prefix issa:   <http://data-issa.cirad.fr/>
%prefix issapr: <http://data-issa.cirad.fr/property/>
%prefix oa:     <http://www.w3.org/ns/oa#>
%prefix prov:   <http://www.w3.org/ns/prov#>
%prefix schema: <http://schema.org/>
%prefix skos:   <http://www.w3.org/2004/02/skos/core#> 
%prefix skosxl: <http://www.w3.org/2008/05/skos-xl#>

%prefix gn:     <http://www.geonames.org/ontology#>
%prefix geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#>
%prefix spatial: <http://jena.apache.org/spatial#>
%prefix units:  <http://www.opengis.net/def/uom/OGC/1.0/>

Endpoint set to: https://data-issa.cirad.fr/sparql
Display: table
Result maximum size: 20
Prefix set: rdfs: = <http://www.w3.org/2000/01/rdf-schema#>
Prefix set: owl: = <http://www.w3.org/2002/07/owl#>
Prefix set: xsd: = <http://www.w3.org/2001/XMLSchema#>
Prefix set: agrovoc: = <http://aims.fao.org/aos/agrovoc/>
Prefix set: dce: = <http://purl.org/dc/elements/1.1/>
Prefix set: dct: = <http://purl.org/dc/terms/>
Prefix set: issa: = <http://data-issa.cirad.fr/>
Prefix set: issapr: = <http://data-issa.cirad.fr/property/>
Prefix set: oa: = <http://www.w3.org/ns/oa#>
Prefix set: prov: = <http://www.w3.org/ns/prov#>
Prefix set: schema: = <http://schema.org/>
Prefix set: skos: = <http://www.w3.org/2004/02/skos/core#>
Prefix set: skosxl: = <http://www.w3.org/2008/05/skos-xl#>
Prefix set: gn: = <http://www.geonames.org/ontology#>
Prefix set: geo: = <http://www.w3.org/2003/01/geo/wgs84_pos#>
Prefix set: spatial: = <http://jena.apache.org/spatial#>
Prefix set: units: = <http://www.opengis.net/def/uom/OGC/1.0/>

____

# Geographic named entities

### 1. Get the geographic NEs of a specific article

Look for the geographic named entities of article http://data-issa.cirad.fr/article/455916, and retrieve their details from GeoNames (stored locally in named graph http://geonames.org/graph).

The alternate names of the entity are concatenated in variable ?alternateNames.

In [2]:
SELECT ?geoEntity ?name ?latitude ?longitude ?altitude ?officialName (group_concat(distinct ?alternateName ; separator=", ") as ?alternateNames) ?nameParentCountry
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
WHERE {
    []  a issa:GeographicAnnotation;
        schema:about <http://data-issa.cirad.fr/article/455916>;
        oa:hasBody ?geoEntity.
    
    ?geoEntity
        gn:name   ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
        OPTIONAL {
            ?geoEntity gn:officialName ?officialName
            FILTER langMatches(lang(?officialName), "en")
        }.
        OPTIONAL {
            ?geoEntity gn:alternateName ?alternateName.
            FILTER langMatches(lang(?alternateName), "en")
        }.
        OPTIONAL { ?geoEntity geo:alt ?altitude }.
        OPTIONAL { ?geoEntity gn:parentCountry  [ gn:name ?nameParentCountry ] }.
} group by ?geoEntity ?name ?latitude ?longitude ?altitude ?officialName ?nameParentCountry

geoEntity,name,latitude,longitude,altitude,officialName,alternateNames,nameParentCountry
https://sws.geonames.org/124544/,Māzandarān,36.25,52.33333,,Mazandaran,Māzandarān Province,Iran
https://sws.geonames.org/132892/,Gorgan,36.8427,54.44391,,,Gorgan,Iran
https://sws.geonames.org/130758/,Iran,32,53,,Iran,"Empire of Iran, Iran, Islamic Republic Of, Islamic Republic of Iran",


### 2.1 Find the geographic NEs of articles with descriptor "cacao"

Look for the articles that have the Agrovoc "cacao" concept (http://aims.fao.org/aos/agrovoc/c_7713) as a descriptor.

Then get the geographic named entities of those articles and retrieve their details from GeoNames.

In [3]:
SELECT distinct ?geoEntity ?name ?latitude ?longitude
FROM <http://data-issa.cirad.fr/graph/articles>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?article;
        oa:hasBody agrovoc:c_7713. # Concept "cacao"

    []  a issa:GeographicAnnotation;
        schema:about ?article;
        oa:hasBody ?geoEntity.

    ?geoEntity
        gn:name         ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
} limit 10

geoEntity,name,latitude,longitude
https://sws.geonames.org/7729885/,Western Africa,18.31281,-4.83398
https://sws.geonames.org/2233387/,Cameroon,6,12.5
https://sws.geonames.org/2300660/,Ghana,8.1,-1.2
https://sws.geonames.org/4372505/,WMDT-TV (Salisbury),38.50178,-75.73549
https://sws.geonames.org/2410763/,São Tomé,0.33654,6.72732
https://sws.geonames.org/1644615/,Pulau Galang,0.75,104.23333
https://sws.geonames.org/1562822/,Vietnam,16.16667,107.83333
https://sws.geonames.org/1814991/,China,35,105
https://sws.geonames.org/2542007/,Morocco,32,-6
https://sws.geonames.org/2596935/,Soubré,5.78377,-6.59388


### 2.2 Find the geographic NEs of articles with descriptor "cacao", with the articles

Similar to the previous query but also return the articles.

In [141]:
SELECT distinct ?article ?articleVisu ?geoEntity ?name ?latitude ?longitude ?altitude
FROM <http://data-issa.cirad.fr/graph/articles>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?article;
        oa:hasBody agrovoc:c_7713. # Concept "cacao"

    []  a issa:GeographicAnnotation;
        schema:about ?article;
        oa:hasBody ?geoEntity.

    ?geoEntity
        gn:name         ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
    
    ?article dct:identifier ?id.
    bind(iri(concat("http://issa.i3s.unice.fr/visu/?uri=", ?article)) as ?articleVisu)
} limit 10

article,articleVisu,geoEntity,name,latitude,longitude,altitude
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/3017382/,France,46,2,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/3469034/,Brazil,-10,-55,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/2635167/,United Kingdom,54.75844,-2.69531,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/3624060/,Costa Rica,10,-84,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/3658394/,Ecuador,-1.25,-78.25,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/2646057/,Ipswich,52.05917,1.15545,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/2134431/,Vanuatu,-16,167,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/4350359/,Capitol Heights,38.88511,-76.91581,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/1733045/,Malaysia,2.5,112.5,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/6252001/,United States,39.76,-98.5,


### 3. Find the articles with geographic NEs within a certain distance of a given point

In [142]:
SELECT distinct ?article ?articleVisu
FROM <http://data-issa.cirad.fr/graph/articles>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
WHERE {
    []  a issa:GeographicAnnotation;
        schema:about ?article;
        oa:hasBody ?geoEntity.

    ?geoEntity
        gn:name   ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
    
    FILTER (bif:st_within(bif:st_point(xsd:float(?longitude), xsd:float(?latitude)), bif:st_point (8, 10), 2000))
    
    ?article dct:identifier ?id.
    bind(iri(concat("http://issa.i3s.unice.fr/visu/?uri=", ?article)) as ?articleVisu)
} LIMIT 10

article,articleVisu
http://data-issa.cirad.fr/article/455935,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/455935
http://data-issa.cirad.fr/article/433785,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/433785
http://data-issa.cirad.fr/article/395431,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/395431
http://data-issa.cirad.fr/article/454957,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/454957
http://data-issa.cirad.fr/article/455868,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/455868
http://data-issa.cirad.fr/article/394049,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/394049
http://data-issa.cirad.fr/article/410537,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/410537
http://data-issa.cirad.fr/article/455078,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/455078
http://data-issa.cirad.fr/article/389182,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/389182
http://data-issa.cirad.fr/article/599920,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/599920


____

# Search by descriptors

### 4. Find all articles' descriptors, ordered by number of occurrences (nb of articles that have that descriptor)

In [22]:
%show 50

SELECT distinct ?concept ?label (count(?concept) as ?count)
FROM <http://agrovoc.fao.org/graph>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasBody ?concept.
    ?concept  skosxl:prefLabel [ skosxl:literalForm ?label ].
    
    filter (langMatches(lang(?label), "fr"))
} group by ?concept ?label order by desc(?count)

Result maximum size: 50

concept,label,count
http://aims.fao.org/aos/agrovoc/c_3081,France,3640
http://aims.fao.org/aos/agrovoc/c_1229,Cameroun,1368
http://aims.fao.org/aos/agrovoc/c_4027,Côte d'Ivoire,1277
http://aims.fao.org/aos/agrovoc/c_6970,Sénégal,1121
http://aims.fao.org/aos/agrovoc/c_1070,Brésil,1072
http://aims.fao.org/aos/agrovoc/c_4510,Madagascar,1065
http://aims.fao.org/aos/agrovoc/c_8355,Afrique occidentale,986
http://aims.fao.org/aos/agrovoc/c_806,banane,910
http://aims.fao.org/aos/agrovoc/c_8081,Burkina Faso,881
http://aims.fao.org/aos/agrovoc/c_6543,La Réunion,842


### 5. Find all articles' descriptors in a subtree of concepts, ordered by number of occurrences (nb of articles that have that descriptor)

In [30]:
%show 50

SELECT distinct ?concept ?label (count(?concept) as ?count)
FROM <http://agrovoc.fao.org/graph>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasBody ?concept.

    ?concept  skosxl:prefLabel [ skosxl:literalForm ?label ].
    filter (langMatches(lang(?label), "fr"))
    
    ?concept skos:broader* agrovoc:c_201. # all concepts "politique agricole" or beneath
    
} group by ?concept ?label order by desc(?count)

Result maximum size: 50

concept,label,count
http://aims.fao.org/aos/agrovoc/c_10967,sécurité alimentaire,382
http://aims.fao.org/aos/agrovoc/c_201,politique agricole,373
http://aims.fao.org/aos/agrovoc/c_195,politique foncière,104
http://aims.fao.org/aos/agrovoc/c_4178,réforme foncière,64
http://aims.fao.org/aos/agrovoc/c_196,réforme agraire,22
http://aims.fao.org/aos/agrovoc/c_9000078,sécurité alimentaire des ménages,12
http://aims.fao.org/aos/agrovoc/c_7d40c88b,accès à la nourriture,8
http://aims.fao.org/aos/agrovoc/c_4173,remembrement,2
http://aims.fao.org/aos/agrovoc/c_57ba042c,sécurité nutritionnelle,1
http://aims.fao.org/aos/agrovoc/c_37734,distribution des terres,1


____

# Search by country

### 6. Count articles with descriptor "cacao", that mention places in some given countries

In [84]:
%show 50

SELECT (count(distinct ?article) as ?count) ?countryName
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/articles>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    # ----- Crops
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?article;
        oa:hasBody agrovoc:c_7713. # Concept "cacao"

    
    # ----- Geographic entities
    ?geoAnnot
        a issa:GeographicAnnotation;
        schema:about ?article.
    
    # Either the geo annotation points to Country or any of its subterritories
    { ?geoAnnot oa:hasBody ?country }
    UNION             
    { ?geoAnnot oa:hasBody [ gn:parentCountry ?country ]. }

    # Considered countries
    ?country gn:name ?countryName.
    values ?country { 
        <https://sws.geonames.org/3017382/> # France
        <https://sws.geonames.org/3469034/> # Brazil
        <https://sws.geonames.org/3624060/> # Costa Rica
        <https://sws.geonames.org/3658394/> # Ecuador
        <https://sws.geonames.org/2233387/> # Cameroon
        <https://sws.geonames.org/2300660/> # Ghana
    }
    
} order by ?countryName

Result maximum size: 50

count,countryName
48,Brazil
51,Cameroon
13,Costa Rica
25,Ecuador
35,France
37,Ghana


### 7. Count articles with descriptors in some given crop caterogies, that mention places in some given countries

In [73]:
%show 50

SELECT (count(distinct ?article) as ?count) ?cropName ?countryName
FROM <http://agrovoc.fao.org/graph>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/articles>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    # ----- Crops
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?article;
        oa:hasBody ?crop.

    { ?crop skos:broader+ agrovoc:c_1474 } # cereals
    UNION
    { agrovoc:c_7974 <http://aims.fao.org/aos/agrontology#includes> ?crop } # <fruits tropicaux> includes ...
             
    ?crop skosxl:prefLabel [ skosxl:literalForm ?cropName ].
    filter (langMatches(lang(?cropName), "fr"))


    # ----- Geographic entities
    ?geoAnnot
        a issa:GeographicAnnotation;
        schema:about ?article.
    
    # Either the geo annotation points to Country or any of its subterritories
    { ?geoAnnot oa:hasBody ?country }
    UNION             
    { ?geoAnnot oa:hasBody [ gn:parentCountry ?country ]. }

    # Considered countries
    ?country gn:name ?countryName.
    values ?country {
        <https://sws.geonames.org/3017382/> # France
        <https://sws.geonames.org/3469034/> # Brazil
        <https://sws.geonames.org/3624060/> # Costa Rica
        <https://sws.geonames.org/3658394/> # Ecuador
        <https://sws.geonames.org/2233387/> # Cameroon
        <https://sws.geonames.org/2300660/> # Ghana
    }
    
} order by ?cropName desc(?count)

Result maximum size: 50

count,cropName,countryName
1,Annona,Brazil
1,Annona,Cameroon
196,agrume,France
102,agrume,Brazil
54,agrume,Ecuador
53,agrume,Cameroon
53,agrume,Costa Rica
28,agrume,Ghana
91,ananas (fruits),France
67,ananas (fruits),Brazil


### 8. Count articles with descriptors in some given crop caterogies, that mention places in some given countries

In [83]:
%show 50

SELECT (count(distinct ?article) as ?count) ?cropName ?countryName
FROM <http://agrovoc.fao.org/graph>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/articles>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    # ----- Crops
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?article;
        oa:hasBody ?crop.

    values ?crop {
        # Tropical fruits
        <http://aims.fao.org/aos/agrovoc/c_13394> # ananas (fruits)
        <http://aims.fao.org/aos/agrovoc/c_9022>  # avocat
        <http://aims.fao.org/aos/agrovoc/c_806>   # banane
        <http://aims.fao.org/aos/agrovoc/c_7713>  # cacao
        <http://aims.fao.org/aos/agrovoc/c_12367> # mangue
        
        # Cereals
        <http://aims.fao.org/aos/agrovoc/c_8373>  # blé
        <http://aims.fao.org/aos/agrovoc/c_12332> # maïs
        <http://aims.fao.org/aos/agrovoc/c_4838>  # millet
        <http://aims.fao.org/aos/agrovoc/c_6599>  # riz
        <http://aims.fao.org/aos/agrovoc/c_7249>  # sorgho en grain
    }

    ?crop skosxl:prefLabel [ skosxl:literalForm ?cropName ].
    filter (langMatches(lang(?cropName), "fr"))


    # ----- Geographic entities
    ?geoAnnot
        a issa:GeographicAnnotation;
        schema:about ?article.
    
    # Either the geo annotation points to Country or any of its subterritories
    { ?geoAnnot oa:hasBody ?country }
    UNION             
    { ?geoAnnot oa:hasBody [ gn:parentCountry ?country ]. }

    # Considered countries
    ?country gn:name ?countryName.
    values ?country {
        <https://sws.geonames.org/3017382/> # France
        <https://sws.geonames.org/3469034/> # Brazil
        <https://sws.geonames.org/3624060/> # Costa Rica
        <https://sws.geonames.org/3658394/> # Ecuador
        <https://sws.geonames.org/2233387/> # Cameroon
        <https://sws.geonames.org/2300660/> # Ghana
    }
    
} order by ?cropName desc(?count)

Result maximum size: 50

count,cropName,countryName
51,Theobroma cacao,Cameroon
48,Theobroma cacao,Brazil
37,Theobroma cacao,Ghana
35,Theobroma cacao,France
25,Theobroma cacao,Ecuador
13,Theobroma cacao,Costa Rica
91,ananas (fruits),France
67,ananas (fruits),Brazil
62,ananas (fruits),Costa Rica
58,ananas (fruits),Ecuador


### 8b. Same as above but lists all the articles instead of count them

In [9]:
%show 50

SELECT distinct ?article ?title ?date ?abstract ?countryName
FROM <http://agrovoc.fao.org/graph>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/articles>
FROM <http://data-issa.cirad.fr/graph/articles/text>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    # ----- Crops
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?article;
        oa:hasBody ?crop.

    values ?crop {
        # Tropical fruits
        <http://aims.fao.org/aos/agrovoc/c_13394> # ananas (fruits)
        <http://aims.fao.org/aos/agrovoc/c_9022>  # avocat
        <http://aims.fao.org/aos/agrovoc/c_806>   # banane
        <http://aims.fao.org/aos/agrovoc/c_7713>  # cacao
        <http://aims.fao.org/aos/agrovoc/c_12367> # mangue
        
        # Cereals
        <http://aims.fao.org/aos/agrovoc/c_8373>  # blé
        <http://aims.fao.org/aos/agrovoc/c_12332> # maïs
        <http://aims.fao.org/aos/agrovoc/c_4838>  # millet
        <http://aims.fao.org/aos/agrovoc/c_6599>  # riz
        <http://aims.fao.org/aos/agrovoc/c_7249>  # sorgho en grain
    }

    ?crop skosxl:prefLabel [ skosxl:literalForm ?cropName ].
    filter (langMatches(lang(?cropName), "fr"))

    
    # ----- Geographic entities
    ?geoAnnot
        a issa:GeographicAnnotation;
        schema:about ?article.
    
    # Either the geo annotation points to Country or any of its subterritories
    { ?geoAnnot oa:hasBody ?country }
    UNION             
    { ?geoAnnot oa:hasBody [ gn:parentCountry ?country ]. }

    # Considered countries
    ?country gn:name ?countryName.
    values ?country {
        <https://sws.geonames.org/3017382/> # France
        <https://sws.geonames.org/3469034/> # Brazil
        <https://sws.geonames.org/3624060/> # Costa Rica
        <https://sws.geonames.org/3658394/> # Ecuador
        <https://sws.geonames.org/2233387/> # Cameroon
        <https://sws.geonames.org/2300660/> # Ghana
    }


    # ---- Articles details
    ?article dct:title ?title ; dct:issued ?date ; dct:abstract [ rdf:value ?abstract ].
    
} order by ?countryName

Result maximum size: 50

article,title,date,abstract,countryName
http://data-issa.cirad.fr/article/388969,Les cacaoyers spontanés de Guyane. Nouvelles prospections,1997,"L'article relate de nouvelles prospections de cacaoyers spontanés, en 1995, dans le sud-est de la Guyane française. Deux nouvelles zones ont été étudiées, les berges des rivières Euleupousing et Yaloupi, affluents du haut-Oyapok, et quelques prélèvements ont été effectués le long du fleuve lui-même. Trois populations ont été identifiées. Le matériel collecté est décrit (localisation, écologie, morphologie). Des caractéristiques nouvelles ou rares du matériel spontané guyanais sont révélées, comme les fèves claires et la forme Calabacillo. L'absence totale de symptômes de balai de sorcière est confirmée. La discussion apporte des arguments en faveur d'une complète distinction entre cacaoyers guyanais et amazoniens",Brazil
http://data-issa.cirad.fr/article/573379,Production durable de cacao : s'inspirer de l'agroforesterie,2014,"Pour satisfaire la consommation croissante de produits chocolatés, de nombreuses initiatives ont été lancées depuis une décennie afin d'augmenter la production de cacao. Ces initiatives continuent de promouvoir le modèle intensif en intrants préconisé depuis les années 1960, alors que ce modèle a atteint ses limites agronomiques, socio-économiques et environnementales. D'où la proposition de s'inspirer de l'agroforesterie pour faire évoluer le modèle actuel : introduire des arbres fruitiers et forestiers dans les cacaoyères contribue à l'intensification agroécologique de la cacaoculture, tout en apportant souplesse et résilience, nécessaires aux petits agriculteurs, qui produisent 95 % du cacao mondial.",Brazil
http://data-issa.cirad.fr/article/387761,L'évolution de la production cotonnière bolivienne,1995,"La Bolivie est un petit producteur de coton qui, pour des raisons climatiques et techniques, a vu sa production fluctuer fortement en quarante ans de culture. Actuellement, le cotonnier s'intègre dans un système de production à base de soja et de blé. Sa culture est en grande partie mécanisée, mais grâce à la main-d'oeuvre employée pour la récolte, elle joue un rôle social important. Une grande proportion de la fibre est exportée vers les pays voisins où elle est appréciée pour sa qualité. L'évolution de cette filière est conditionnée en particulier par la recherche de variétés résistantes aux maladies et par le développement des exportations vers d'autres pays tels que la Chine, le Brésil, le Chili",Brazil
http://data-issa.cirad.fr/article/488763,%22La libéralisation de la filière cacaoyère ivoirienne et les recompositions du marché mondial du cacao : vers la fin des %22%22pays producteurs%22%22 et du marché international ?%22,2001,"The restructuring of the world cocoa market has concluded with the liberalisation of the sector in the world's leading producing country - Côte d'Ivoire - clearing the way for domination by an oligopoly of global companies. This paper describes how Côte d'Ivoire's share of world production created an illusion but not the reality of market power. In the 1990s, in the wake of failed attempts to influence the world market, the Ivorian cocoa experienced a series of upheavals that were both pivotal to broader changes in the global market and a reflection of them. The converging strategies of new Ivorian firms and of the major global grinding companies resulted in increased vertical integration in Côte d'Ivoire, exemplified in the development of 'origin grinding'. Later, financial difficulties encountered by Ivorian firms led to global companies taking control. Amongst the results of these changes ore a decline in the role of traders, a redefinition of relationship between grinders and chocolate manufacturers, and a standardisation of cocoa quality around an average 'bulk' level. This signals the end of 'the producing countries' and of the global market.",Brazil
http://data-issa.cirad.fr/article/388091,Le cacaoyer: stratégies de sélect